In [10]:
# Load environment variables from .env file
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# Create Graph with a state of all messages

In [11]:
from langgraph.graph import add_messages, StateGraph
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.checkpoint.sqlite import SqliteSaver

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    event_type: str
    location: str
    labels: str
    
graph_builder = StateGraph(State)

# Connection to Couchbase's vector store

In [12]:
from langchain_openai import OpenAIEmbeddings
import os
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions
from couchbase.auth import PasswordAuthenticator
from langchain_couchbase import CouchbaseVectorStore

COUCHBASE_CONNECTION_STRING = os.environ["COUCHBASE_CONNECTION_STRING"]
COUCH_USER = os.environ["COUCHBASE_USER"]
COUCH_PASS = os.environ["COUCHBASE_PASS"]
BUCKET_NAME = os.environ["COUCHBASE_BUCKET"]
SCOPE_NAME = os.environ["COUCHBASE_SCOPE"]
COLLECTION_NAME = os.environ["COUCHBASE_COLLECTION"]
SEARCH_INDEX_NAME = os.environ["COUCHBASE_SEARCH_INDEX"]

auth = PasswordAuthenticator(COUCH_USER, COUCH_PASS)
options = ClusterOptions(auth)
cluster = Cluster(COUCHBASE_CONNECTION_STRING, options)
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

vector_store = CouchbaseVectorStore(
    cluster=cluster,
    bucket_name=BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embedding,
    index_name=SEARCH_INDEX_NAME,
)


# Create Graph instance

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
template = """You are a helpful bot that serves the purpose of finding events for artists looking for venues in the USA. If you cannot answer based on the context provided, respond with a generic
    answer. Answer the question as truthfully as possible using the context below: {context}
Please also format the result in Markdown format.
            Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

generation_chain = prompt | llm | StrOutputParser()


def chatbot(state: State):
    response = generation_chain.invoke({"context": state['messages'], "question": f"We are a {state['event_type']} amateur group looking for {state['labels']} festivals in {state['location']}, can you please recommend some for us to reach out to?"})
    state['messages'].append(response)
    return state

def search_couchbase(state: State):
    query = f"Geo Info: {state['location']}"
    
    retriever = vector_store.as_retriever()
    results = retriever.invoke(query)
    for result in results:
        text = f"Title: {result.metadata['title']}/{result.metadata['alternate_titles_flat']} - {result.metadata['description']} from {result.metadata['start']} to {result.metadata['end']}, location {result.metadata['geo_info']}. Labels {result.metadata['labels_flat']}, category {result.metadata['category']}"
        state['messages'].append(text)
    return state

graph_builder.add_node("vector_search", search_couchbase)
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("vector_search")
graph_builder.add_edge("vector_search", "chatbot")
graph_builder.set_finish_point("chatbot")

memory = SqliteSaver.from_conn_string(":memory:")
graph = graph_builder.compile(checkpointer=memory)


# Interaction

In [14]:
from random import randint
from IPython.core.display import Markdown

session_id = randint(1, 10000)
config = {"configurable": {"thread_id": session_id}}

input_location = "kansas"
input_category = "jaz"
input_labels = "grange"

# Stream the graph, each output will be printed when ready
for event in graph.stream({"event_type": input_category, "location": input_location, "labels": input_labels}, config):
    for value in event.values():
        if len(value['messages']) > 0:
            display(Markdown(value['messages'][-1]))

Title: For King & Country/For King & Country at T-Mobile Center -  from 2024-10-07T00:00:00Z to 2024-10-07T00:00:00Z, location 1407 Grand Blvd, Kansas City, MO 64106, USA. Labels concert, music, category concerts

### Events in Kansas City, MO

1. **Sal Vulcano (Theater)**
   - **Date:** October 5, 2024
   - **Time:** 12:00 AM to 2:15 AM
   - **Location:** Kansas City, MO, USA
   - **Labels:** Comedy, Entertainment, Performing Arts
   - **Category:** Performing Arts

2. **The Marshall Tucker Band/Marshall Tucker Band and Jefferson Starship**
   - **Date:** October 5, 2024
   - **Time:** 12:30 AM
   - **Location:** 300 W 12th St, Kansas City, MO 64105, USA
   - **Labels:** Concert, Music
   - **Category:** Concerts

3. **Taylor Shines Laser Spectacular at Grinder's**
   - **Date:** October 6, 2024
   - **Time:** 12:30 AM
   - **Location:** 419 E 18th St, Kansas City, MO 64108, USA
   - **Labels:** Concert, Music
   - **Category:** Concerts

4. **For King & Country/For King & Country at T-Mobile Center**
   - **Date:** October 7, 2024
   - **Time:** 12:00 AM
   - **Location:** 1407 Grand Blvd, Kansas City, MO 64106, USA
   - **Labels:** Concert, Music
   - **Category:** Concerts

Unfortunately, there are no specific jazz festivals mentioned in the provided events. You may want to explore local event listings or contact venues directly for more information on jazz festival opportunities in Kansas City.